In [3]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from tqdm import tqdm

%matplotlib inline

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
def image_shower(images, labels, n=4):
    plt.figure(figsize=(12, 12))
    for i, image in enumerate(images[:n]):
        plt.subplot(n, n, i + 1)
        image = image/ 2 + 0.5
        plt.imshow(image.numpy().transpose((1, 2, 0)).squeeze())
    print("Real Labels: ", ' '.join('%5s' % classes[label] for label in labels[:n]))
classes = ("gothic", "muslim", "orthodox")

PATH = "C:\Users\Admin\ML.COURSE\CV\Homework\different_temples_dataset"

In [6]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
validation_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [7]:
train_dataset = torchvision.datasets.ImageFolder(os.path.join(PATH, "train"), transform=train_transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, num_workers=0, shuffle=True)

test_dataset = torchvision.datasets.ImageFolder(os.path.join(PATH, "test"), transform=test_transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, num_workers=0, shuffle=True)

validation_dataset = torchvision.datasets.ImageFolder(os.path.join(PATH, "validation"), transform=validation_transform)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=64, num_workers = 0, shuffle = True)

OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'https://fex.net/ru/s/pxtmzxk\\train'

In [ ]:
images, labels = next(iter(train_loader))
image_shower(images, labels)

In [8]:
class MyCNN(nn.Module):
    def __init__(self, num_classes):
        super(MyCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(256 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(-1, 256 * 28 * 28)  
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [9]:

model = MyCNN(3)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.to(device)

MyCNN(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=200704, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=3, bias=True)
)

In [11]:
train_losses = []
test_losses = []
train_accuracies = []
test_accuracies = []

In [13]:
epochs = 7
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100 * correct / total
    train_losses.append(running_loss / len(train_loader))
    
    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {running_loss/len(train_loader):.4f}, Training Accuracy: {train_accuracy:.2f}%")

Epoch 1/7, Training Loss: 1.8840, Training Accuracy: 42.71%
Epoch 2/7, Training Loss: 0.8326, Training Accuracy: 62.09%
Epoch 3/7, Training Loss: 0.7131, Training Accuracy: 70.39%
Epoch 4/7, Training Loss: 0.6108, Training Accuracy: 75.36%
Epoch 5/7, Training Loss: 0.5223, Training Accuracy: 79.29%
Epoch 6/7, Training Loss: 0.4493, Training Accuracy: 82.69%
Epoch 7/7, Training Loss: 0.3978, Training Accuracy: 84.73%


In [ ]:
model.eval()
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
test_losses.append(test_loss / len(test_loader))

print(f"Epoch {epoch + 1}/{epochs}, Testing Loss: {test_loss/len(test_loader):.4f}, Testing Accuracy: {test_accuracy:.2f}%")

In [ ]:
plt.plot(train_losses, label='Training Loss')
plt.plot(test_losses, label='Test Loss')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Over Epochs')
plt.show()

plt.plot(train_accuracies, label='Training Accuracy')
plt.plot(test_accuracies, label='Test Accuracy')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Over Epochs')
plt.show()

In [ ]:
images, labels = next(iter(testloader))
image_shower(images, labels)

outputs = model(images.to(device))

_, predicted = torch.max(outputs, 1)

print("Predicted: ", " ".join("%5s" %classes[predict] for predict in predicted[:4]))